# 🌦️ Analyse Météo – Pipeline Landing → Staging → Refined → Warehouse

Ce notebook illustre la partie **analyse / visualisation** d'une mini-pipeline Data :

- Données générées (CSV) → `data/raw/`
- Nettoyage (JSON) → `data/staging/`
- Format optimisé (Parquet) → `data/refined/`
- Entrepôt analytique (DuckDB) → `warehouse/weather.duckdb`

L'objectif est de montrer une analyse professionnelle, reproductible, basée sur la table **`weather`** stockée dans DuckDB.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Configuration chemins
DB_PATH = "../warehouse/weather.duckdb"
TABLE_NAME = "weather"

con = duckdb.connect(DB_PATH)

# Chargement complet (pour analyse) – trié par date
df = con.execute(f"SELECT * FROM {TABLE_NAME} ORDER BY date").df()

df.head()

## 📋 Aperçu du dataset

- Une ligne = une date (jour)
- Variables principales :
  - `city` : ville
  - `temperature_c` : température moyenne (°C)
  - `humidity` : humidité (%)
  - `wind_kmh` : vitesse du vent (km/h)
  - `precip_mm` : précipitations (mm)
  - `is_storm` : indicateur d'épisode orageux (0/1)


In [ ]:
print("Dimensions du dataset :", df.shape)
print("\nTypes de données :")
print(df.dtypes)

print("\nStatistiques descriptives (numériques) :")
display(df.describe())

## 🌍 Répartition par ville


In [ ]:
city_counts = df["city"].value_counts()
print(city_counts)

plt.figure(figsize=(6,4))
city_counts.plot(kind="bar")
plt.title("Nombre de lignes par ville")
plt.xlabel("Ville")
plt.ylabel("Nombre d'observations")
plt.tight_layout()
plt.show()

## 🌦️ Distribution des variables météo


In [ ]:
numeric_cols = ["temperature_c", "humidity", "wind_kmh", "precip_mm"]

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()

for ax, col in zip(axes, numeric_cols):
    ax.hist(df[col].dropna(), bins=30)
    ax.set_title(col)

plt.tight_layout()
plt.show()

## ⛈️ Taux d'événements orageux


In [ ]:
storm_rate = df["is_storm"].mean()
print(f"Taux global d'épisodes orageux : {storm_rate:.2%}")

storm_by_city = df.groupby("city")["is_storm"].mean().sort_values(ascending=False)
display(storm_by_city)

plt.figure(figsize=(6,4))
storm_by_city.plot(kind="bar")
plt.title("Taux d'orages par ville")
plt.ylabel("Taux d'orages")
plt.tight_layout()
plt.show()

## 📈 Température dans le temps


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df["date"], df["temperature_c"])
plt.title("Température au fil du temps (toutes villes)")
plt.xlabel("Date")
plt.ylabel("Température (°C)")
plt.tight_layout()
plt.show()

### Température moyenne par ville


In [ ]:
temp_by_city = df.groupby("city")["temperature_c"].mean().sort_values()
display(temp_by_city)

plt.figure(figsize=(6,4))
temp_by_city.plot(kind="bar")
plt.title("Température moyenne par ville")
plt.ylabel("Température (°C)")
plt.tight_layout()
plt.show()

## 🔗 Corrélations entre variables météo


In [ ]:
corr = df[["temperature_c", "humidity", "wind_kmh", "precip_mm", "is_storm"]].corr()
print(corr)

fig, ax = plt.subplots(figsize=(6,5))
im = ax.imshow(corr.values)

ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha="right")
ax.set_yticklabels(corr.columns)
plt.title("Matrice de corrélation")
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

## 🧮 Exemple de requêtes SQL DuckDB

Ces exemples illustrent comment exploiter l'entrepôt DuckDB directement en SQL, comme on le ferait avec Athena sur un Data Lake AWS.

In [ ]:
# Température moyenne par ville (SQL)
sql_temp_by_city = con.execute(
    """
    SELECT city, AVG(temperature_c) AS avg_temp
    FROM weather
    GROUP BY city
    ORDER BY avg_temp
    """
).df()

sql_temp_by_city

In [ ]:
# Taux d'orages par ville (SQL)
sql_storm_by_city = con.execute(
    """
    SELECT city, AVG(is_storm)::DOUBLE AS storm_rate
    FROM weather
    GROUP BY city
    ORDER BY storm_rate DESC
    """
).df()

sql_storm_by_city

## 🧾 Synthèse des insights (exemple de restitution métier)

- **Qualité des données** :
  - Série temporelle continue sur la période simulée
  - Variables météo bien typées et sans valeurs extrêmes incohérentes

- **Climat global** :
  - Température moyenne, humidité et vent dans des plages réalistes
  - Variabilité visible sur la série temporelle

- **Orages** :
  - Un certain pourcentage de jours est marqué comme `is_storm = 1`
  - Ce taux peut être comparé entre villes

- **Corrélations** :
  - On peut analyser les liens entre vent, précipitations et orages
  - Ce type de structure est exploitable pour un futur modèle de prédiction d'événements météo

Ce notebook constitue une base pour :
- des analyses exploratoires plus poussées,
- des cas d'usage prédictifs (prévision d'orages),
- ou une extension vers une vraie architecture Cloud (S3 / Glue / Athena / QuickSight).